<a href="https://colab.research.google.com/github/felipedram/xf-models-analysis/blob/master/trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Funções para teste](http://delta.cs.cinvestav.mx/~ccoello/EMOO/testfuncs/)<br>
[Documentação basin-hopping](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html#scipy.optimize.basinhopping)<br>
[Documentação minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)<br>
[Basin-Hopping Paper](https://arxiv.org/pdf/cond-mat/9803344.pdf)<br>
[Stochastic Gradient Descent Towards Data Science](https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f)<br>
[Stochastic Gradient Descent](https://www.pyimagesearch.com/2016/10/17/stochastic-gradient-descent-sgd-with-python/)
[StatQuest Gradient Descent](https://www.youtube.com/watch?v=sDv4f4s2SB8)<br>
[StatQuest Stochastic Gradient Descent](https://www.youtube.com/watch?v=vMh0zPT0tLI)

In [0]:
import numpy as np
import scipy.optimize as opt

In [0]:
class f(object):
    def __init__(self, solver, goal):
        self.__solver = solver
        if goal == "max" or goal == "min":
            self.__goal = goal
        else:
            raise Exception("goal deve ser 'max' ou 'min'.")
        self.__f_max = None
        self.__x_max = None
        self.__f_min = None
        self.__x_min = None

    @property
    def goal(self):
        return self.__goal

    @property
    def f_max(self):
        return self.__f_max

    @f_max.setter
    def f_max(self, f_max):
        self.__f_max = f_max

    @property
    def x_max(self):
        return self.__x_max

    @x_max.setter
    def x_max(self, x_max):
        self.__x_max = x_max

    @property
    def f_min(self):
        return self.__f_min

    @f_max.setter
    def f_min(self, f_min):
        self.__f_min = f_min

    @property
    def x_min(self):
        return self.__x_min

    @x_min.setter
    def x_min(self, x_min):
        self.__x_min = x_min

    def solve(self, x, inv=False):
        x = np.array(x)
        return self.__solver(x) * -1 if inv else self.__solver(x)

    def mu(self, x):
        x = np.array(x)
        if self.__goal == "max":
            return (self.solve(x) - self.__f_min) / (self.__f_max - self.__f_min)
        elif self.__goal == "min":
            return (self.__f_max - self.solve(x)) / (self.__f_max - self.__f_min)

In [0]:
class f_linear(f):
    def __init__(self, coefs, goal):
        self.__coefs = np.array(coefs)
        f.__init__(self, lambda x: np.dot(self.__coefs, x), goal)

    @property
    def coefs(self):
        return self.__coefs

In [0]:
f1 = f_linear([0.06, 0.53, 0.18, 0.18, 0.06], "max")
f2 = f_linear([25, 70, 60, 95, 45], "max")
f3 = f_linear([0, 32.5, 300, 120, 0], "min")
f4 = f_linear([0.1, 0.1, 0.11, 0.35, 0.33], "min")

In [0]:
A_eq = np.array([[1, 1, 1, 1, 1]])
b_eq = np.array([3000])
x0_bounds = (0, 850)
x1_bounds = (0, 220)
x2_bounds = (0, 1300)
x3_bounds = (0, 1615)
x4_bounds = (0, 700)

In [0]:
F = [f1, f2, f3, f4]
bounds = [x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds]

In [0]:
from scipy.optimize import linprog
import warnings

for f_ in F:
    res = linprog(f_.coefs * -1, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    if res.success:
        f_.f_max = -1 * res.fun
        f_.x_max = res.x
    else:
        warnings.warn("Otimização mal sucedida.")

In [0]:
for f_ in F:
    res = linprog(f_.coefs, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    if res.success:
        f_.f_min = res.fun
        f_.x_min = res.x
    else:
        warnings.warn("Otimização mal sucedida.")

In [0]:
def muD_min(x):
    x = np.array(x)
    return np.max([f_.mu(x)*-1 for f_ in F])

In [0]:
def basinhopping_bounds(**kwargs):
    x = kwargs["x_new"]
    resp = True
    if np.dot(x, A_eq[0]) != b_eq[0]:
        resp = False
    if x[0] < x0_bounds[0] or x[0] > x0_bounds[1]:
        resp = False
    if x[1] < x1_bounds[0] or x[1] > x1_bounds[1]:
        resp = False
    if x[2] < x2_bounds[0] or x[2] > x2_bounds[1]:
        resp = False
    if x[3] < x3_bounds[0] or x[3] > x3_bounds[1]:
        resp = False
    if x[4] < x4_bounds[0] or x[4] > x4_bounds[1]:
        resp = False
    return resp



cobyla_constraints = [
    {"type": "ineq", "fun": lambda x: x[0]},
    {"type": "ineq", "fun": lambda x: x0_bounds[1] - x[0]},
    {"type": "ineq", "fun": lambda x: x[1]},
    {"type": "ineq", "fun": lambda x: x1_bounds[1] - x[1]},
    {"type": "ineq", "fun": lambda x: x[2]},
    {"type": "ineq", "fun": lambda x: x2_bounds[1] - x[2]},
    {"type": "ineq", "fun": lambda x: x[3]},
    {"type": "ineq", "fun": lambda x: x3_bounds[1] - x[3]},
    {"type": "ineq", "fun": lambda x: x[4]},
    {"type": "ineq", "fun": lambda x: x4_bounds[1] - x[4]},
    {"type": "eq", "fun": lambda x: np.dot(x, A_eq[0]) - b_eq[0]},
]

In [0]:
minimizer_kwargs = {"method": "SLSQP", "constraints": cobyla_constraints}
res1 = opt.basinhopping(
    muD_min,
    f1.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res2 = opt.basinhopping(
    muD_min,
    f2.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res3 = opt.basinhopping(
    muD_min,
    f3.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res4 = opt.basinhopping(
    muD_min,
    f4.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)

In [32]:
print(res1.fun*-1)
print(res2.fun*-1)
print(res3.fun*-1)
print(res4.fun*-1)


0.0840565860786446
0.08523556576403987
0.5055088038227242
0.3415812179269408


In [0]:
minimizer_kwargs = {"method": "SLSQP", "constraints": cobyla_constraints}
res1 = opt.basinhopping(
    muD_min,
    f1.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res2 = opt.basinhopping(
    muD_min,
    f2.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res3 = opt.basinhopping(
    muD_min,
    f3.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)
res4 = opt.basinhopping(
    muD_min,
    f4.x_max,
    minimizer_kwargs=minimizer_kwargs,
    accept_test=basinhopping_bounds,
    disp=False,
)

In [34]:
print(res1.fun*-1)
print(res2.fun*-1)
print(res3.fun*-1)
print(res4.fun*-1)

0.08389582945566146
0.08502730922110137
0.29916709816486603
0.001810864235456888


In [35]:
pip install --upgrade pyswarm

Requirement already up-to-date: pyswarm in /usr/local/lib/python3.6/dist-packages (0.6)


In [0]:
lb = [b[0] for b in bounds]
ub = [b[1] for b in bounds]
cons = [lambda x: np.dot(A_eq, x)-b_eq[0]+0.5, lambda x: np.dot(A_eq, x)-b_eq[0]-0.5]

In [37]:
from pyswarm import pso

xpt, fopt = pso(muD_min, lb, ub, ieqcons=cons)


Stopping search: Swarm best objective change less than 1e-08


In [38]:
print(xpt)
print(fopt*-1)

[811.02023851 216.46390429 776.23475962 995.01319245 356.02429646]
0.5262826160236658


In [0]:
qtd_x = len(bounds)

x_para_passo_acima = 0
x_para_passo_abaxo = 0

In [66]:
def muD_max(x):
    x = np.array(x)
    return np.min([f_.mu(x) for f_ in F])

posicao = f1.x_max
muD_posicao_atual = muD_max(posicao)
passo = 0.5

i = 0

while i < qtd_x:
    
    if posicao[i] + passo <= bounds[i][1]:
        nova_posicao = posicao[:]
        nova_posicao[i] += passo
        
        j = i+1
        while j < qtd_x:
            if posicao[j] - passo >= bounds[j][0]:
                nova_posicao[j] -= passo
                print('Estamos: {0} \n\t tentamos: {1}'.format(posicao, nova_posicao))
                if muD_max(nova_posicao) > muD_max(posicao):
                    posicao = nova_posicao[:]
                    print('Ok! Avançamos para {1}'.format(posicao))
                    break
                else:
                    nova_posicao[j] += passo
                    print('Negativo! Voltamos para {}'.format(nova_posicao))
                
            j += 1
        
    i += 1
    
print(posicao)        
                

Estamos: [   1.7  219.4 1165.  1615.     0. ] 
	 tentamos: [   1.7  219.4 1165.  1615.     0. ]
Negativo! Voltamos para [   1.7  219.9 1165.  1615.     0. ]
Estamos: [   1.7  219.9 1164.5 1615.     0. ] 
	 tentamos: [   1.7  219.9 1164.5 1615.     0. ]
Negativo! Voltamos para [   1.7  219.9 1165.  1615.     0. ]
Estamos: [   1.7  219.9 1165.  1614.5    0. ] 
	 tentamos: [   1.7  219.9 1165.  1614.5    0. ]
Negativo! Voltamos para [   1.7  219.9 1165.  1615.     0. ]
Estamos: [   1.7  219.9 1165.5 1614.5    0. ] 
	 tentamos: [   1.7  219.9 1165.5 1614.5    0. ]
Negativo! Voltamos para [   1.7  219.9 1165.5 1615.     0. ]
[1.7000e+00 2.1990e+02 1.1655e+03 1.6150e+03 5.0000e-01]


In [59]:
posicao

617.0

In [0]:
while x_para_passo_acima < qtd_x:
    
    if XP[v_up] + step <= upper_bounds[v_up]:
        
        XP[v_up] += step
        XP_down = XP[:]

        while v_down < nvars:
        
            if v_down != v_up:
                if XP_down[v_down]-step >= lower_bounds[v_down]:
                    XP_down[v_down] -= step
                    #Debug:
                    muX  = round(min(MU(X)), 3)
                    muXP = round(min(MU(XP_down)), 3)  
                    print "Estamos: ", X, " | Tentamos: ", XP_down, " : ", muX, " | ", muXP
                    #Fim
                    if min(MU(XP_down)) > min(MU(X)):
                        XP = XP_down[:]
                        X  = XP[:]
                        #Debug:
                        print "melhor"
                        #Fim
                        break
                    else:
                        XP_down = XP[:]
                        #Debug:
                        print "pior"
                        #Fim
            v_down += 1
                
        if sum(XP) > total_bounds:
            XP = X[:]
        else:
            continue
    v_down = 0
    v_up  += 1